In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 100)
%matplotlib inline

# 1 Dataset: documenten_ph
## 1.1 First data exploration
### 1.1.1 Lets have a peek

In [ ]:
pd.read_csv(
    "../data/documenten_ph/DB_V2.TXT",
    encoding="latin_1",
    sep="\t",
    nrows=5,
).head()

### 1.1.2 Lets study the quantity of use over time
For this I only need 3 columns:
* drug_code: to know which drug we are talking about
* realization_date: to know when it was used
* quantity: to know how much of it was used

In [ ]:
df_doc_ph = pd.read_csv(
    "../data/documenten_ph/DB_V2.TXT",
    usecols=["drug_code", "realization_date", "quantity"],
    encoding="latin_1",
    sep="\t",
)

### 1.1.3 Lets visualize this
(We will sample 10 arbitrary drug_codes, otherwise the plot will crash)

In [ ]:
df_doc_ph.groupby(
    ["realization_date", "drug_code"]
).agg(
    {"quantity": "sum"}
).unstack().sample(10, axis=1, random_state=345).plot(marker="o", figsize=(16, 9))

## 1.2 Can we improve the quality of realization_date
### 1.2.1 Cleaning
* there seem to be 5-th quarters, which are wierd cases: Lets get rid of them
* the values are a concatenation (year + quarter): lets make it more smooth

In [ ]:
df_doc_ph = df_doc_ph[
    df_doc_ph.realization_date % 10 != 5
]

df_doc_ph["realization_date_smooth"] = (
    df_doc_ph.realization_date // 10 + df_doc_ph.realization_date % 10 / 4
)

### 1.2.2 Lets see how our plot has improved

In [ ]:
df_doc_ph.groupby(
    ["realization_date_smooth", "drug_code"]
).agg(
    {"quantity": "sum"}
).unstack().sample(10, axis=1, random_state=345).plot(marker="o", figsize=(16, 9))

## 1.3 Can we select a meaningfull subgroup?
We have a secondary data set which contains names of the concerned drugs:
### 1.3.1 Lets have a peek at PHARMA_REF

In [ ]:
pd.read_csv(
    "../data/documenten_ph/PHARMA_REF.csv",
    nrows=5,
    sep=";",
).head()

### 1.3.2 Lets load the data from PHARMA_REF that might help us
* drug_code: to join with the other dataset
* nom_produit_pharma: to have a better recognizable name of the product
* code_atc: such that we can focus on one group of drugs
* DDD: such that we know how many doses this drug is made of

In [ ]:
df_pharma = pd.read_csv(
    "../data/documenten_ph/PHARMA_REF.csv",
    usecols=[
        "drug_code", 
        "nom_produit_pharma",
        "code_atc",
        "DDD",
    ],
    sep=";",
)

### 1.3.3 Lets join the two datasets
* apparently some drug codes in doc_ph are #######
* this also stopped pandas from loading the data as an int64

In [ ]:
df_doc_ph = df_doc_ph[
    df_doc_ph.drug_code != "#######"
]

df_doc_ph.drug_code = df_doc_ph.drug_code.astype("int64")

In [ ]:
df_joined = pd.merge(df_doc_ph, df_pharma, on="drug_code")

#### 1.3.4 Lets focus on Cetirizine an antihistaminica
* 1) https://www.whocc.no/atc_ddd_index/
* 2) search: Cetirizine
* 3) get ATC code: R06AE07

In [ ]:
df_R06AE07 = df_joined[df_joined.code_atc == "R06AE07"]

### 1.3.5 Lets plot the quantities by product name

In [ ]:
df_R06AE07.groupby(
    ["realization_date_smooth", "nom_produit_pharma"]
).agg(
    {"quantity": "sum"}
).unstack().plot(marker="o", figsize=(16, 9))

## 1.4 Whats about quantities vs doses?
Well it turns out that quantities are the number of packages not of doses and DDD seems to be the number of doses in a package

In [ ]:
df_R06AE07["doses"] = (
    pd.to_numeric(df_R06AE07.DDD.str.extract(r'(\d*),0*')[0]) 
    * df_R06AE07.quantity
)

df_R06AE07.groupby(
    ["realization_date_smooth", "nom_produit_pharma"]
).agg(
    {"doses": "sum"}
).unstack().plot(marker="o", figsize=(16, 9))